In [1]:
import networkx as nx
import pandas as pd

In [42]:
uniprot2reactome = pd.read_csv('../data/UniProt2Reactome.txt', sep="\t", names =['input','translation','x','y','z','o'])
uniprot2reactome = uniprot2reactome[uniprot2reactome['o'] == 'Homo sapiens']
uniprot2reactome = uniprot2reactome[['input','translation']]

qm = pd.read_csv('../data/newQM_NONA.csv')
proteins = qm['Protein'].values.tolist()
uniprot2reactome = uniprot2reactome[uniprot2reactome['input'].isin(proteins)]
uniprot2reactome.rename(columns={'input':'child', 'translation':'parent'}, inplace=True)
qm.fillna(0, inplace=True)

In [76]:
reactome_names = pd.read_csv('../data/ReactomePathways.txt', sep ="\t", names = ['accession','trivname','y'])[['trivname','accession']]
uniprot_names = pd.read_csv('../data/human_proteome.gz')[['trivname', 'accession']]
def translate(input_data):
    translate_dict = pd.concat([reactome_names, uniprot_names]).set_index('accession').to_dict()['trivname']
    output_data=input_data.map(translate_dict)
    return output_data



In [86]:
    

def subset_df(sample, qm, uniprot2reactome, reactome_pathways):
    qm_sample = qm[['Protein',sample]]
    qm_sample= qm_sample[qm_sample[sample]> 0]
    
    path_df = uniprot2reactome.merge(qm_sample, left_on='child', right_on='Protein', how='right')
    path_df.dropna(subset=['child'], inplace=True)
    starting_pathways = path_df['parent']
    add_df = reactome_pathways[reactome_pathways['child'].isin(starting_pathways)]
    path_df = pd.concat([path_df, add_df])
    new_pathways = add_df['child']
    i = 0
    while len(new_pathways) > 0:
        i+=1
        add_df = reactome_pathways[reactome_pathways['child'].isin(new_pathways)]
        path_df = pd.concat([path_df, add_df])
        new_pathways = add_df['parent']
    path_df['child'] = translate(path_df['child'])
    path_df['parent'] = translate(path_df['parent'])
    path_df = path_df[path_df['child'] != 0]
    return path_df[['child','parent', sample]].rename(columns={sample:'abundance'}).fillna(0)

    

    
reactome_pathways = pd.read_csv('../data/sepsis_HSA_All_ms_path.csv')
samples = [col for col in qm if col.startswith('TM')]
for sample in samples:
    subset_df(sample, qm, uniprot2reactome, reactome_pathways).to_csv(f'../data/graphs/{sample}.csv', index=False)

In [3]:

df = pd.read_csv('../data/sepsis_HSA_All_ms_path.csv')
df = pd.concat([df, uniprot2reactome])
parent = translate(df['parent'])
child = translate(df['child'])
df = pd.DataFrame({'parent':parent, 'child':child})
df.to_csv('../data/processed/sepsis_edgelist.csv', index=False)

In [4]:
samples = [col for col in qm if col.startswith('TM')]
qm = qm [['Protein'] + samples]
qm['Protein'] = translate(qm['Protein'])

In [14]:
merged = df.merge(qm, left_on='child', right_on='Protein', how='left')
merged = merged[['parent', 'child'] + samples]
print(merged.head())
merged.to_csv('../data/processed/sepsis_edgelist_w_values.csv', index=False)

       parent                                              child  \
0   Apoptosis                    Intrinsic Pathway for Apoptosis   
1   Apoptosis                            Regulation of Apoptosis   
2   Apoptosis  Caspase activation via extrinsic apoptotic sig...   
3   Apoptosis                          Apoptotic execution phase   
4  Hemostasis        Formation of Fibrin Clot (Clotting Cascade)   

   TM_M2012_010  TM_M2012_011  TM_M2012_012  TM_M2012_013  TM_M2012_014  \
0           NaN           NaN           NaN           NaN           NaN   
1           NaN           NaN           NaN           NaN           NaN   
2           NaN           NaN           NaN           NaN           NaN   
3           NaN           NaN           NaN           NaN           NaN   
4           NaN           NaN           NaN           NaN           NaN   

   TM_M2012_016  TM_M2012_017  TM_M2012_018  ...  TM_M2012_190  TM_M2012_191  \
0           NaN           NaN           NaN  ...           N

In [81]:
G = nx.from_pandas_edgelist(df = df, source='parent', target='child', create_using = nx.DiGraph)